# WSW - 40 Knot Club

Iterate through folders, looking for people who have exceeded 40 knots in the harbour.

Copyright 2024 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

## Notes

This is very crude and simply looks for Doppler speeds in excess of 30 m/s (i.e. 108 km/h or 58.32 knots).

Such speeds are currently unrealistic for a windsurfer.

In [1]:
import os
import sys

import numpy as np

from shapely.geometry import Polygon, Point

import traceback

corePath = os.path.join('..', 'core')
if corePath not in sys.path:
    sys.path.extend([corePath])

from file_reader import getFileReader

## Main Function

In [2]:
harbourCoordinates = [
    (50.58302662325741, -2.470407543036802),
    (50.57956801297001, -2.468935848305471),
    (50.57288668557671, -2.463076669924748),
    (50.57135343398436, -2.460734344626166),
    (50.57108144909613, -2.457564965121415),
    (50.57171867189327, -2.455973731673139),
    (50.57089845125837, -2.455031394755522),
    (50.57049960355907, -2.453483849577976),
    (50.57074808724141, -2.453273667205220),
    (50.56927614989874, -2.449277113127991),
    (50.56830919262200, -2.445999974143590),
    (50.56858014912896, -2.443753447607102),
    (50.56769689652297, -2.428783149195592),
    (50.57059840413196, -2.417256198949373),
    (50.57204085515896, -2.416101911293355),
    (50.58487932702268, -2.414610955894832),
    (50.58580125660528, -2.417938240086490),
    (50.59431600604534, -2.431570375679156),
    (50.59621309166282, -2.432731498065036),
    (50.60130077869876, -2.449815610597597),
    (50.59741388890214, -2.459452453376158),
    (50.59506773780025, -2.460824086152496),
    (50.59282114558383, -2.465075005679049),
    (50.58496927287083, -2.470103034761857),
    (50.58302662325741, -2.470407543036802)
]

harbour = Polygon(harbourCoordinates)

speedThreshold = 40 * 1852 / 3600

def findMembers():
    '''Iterate through session archive testing each GPS file'''

    rootDir = os.path.join(projdir, '..', 'wsw-data')

    errors = {}
    stats = {}
    
    totFiles = 0
    totPoints = 0
    totDist = 0

    for root, subDirs, files in os.walk(rootDir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            
            #if ext and ext in ['.sbn', '.sbp', '.ubx']:
            if ext and ext in ['.sbn', '.oao']:
                filePath = os.path.join(root, file)
                reader = getFileReader(filePath)
                try:
                    # Some legacy ESP-GPS files contain bad checksums
                    if ext == '.ubx':
                        reader.load(ignoreChecksums=True)
                    else:
                        reader.load()
                    
                    # Process all tracks within the file
                    for track in reader.tracks:
                        maxSog = 0

                        points = (np.argwhere(track.data['sog'] >= speedThreshold))
                        for i in points:
                            if track.data['hdop'][i[0]] <= 20 and \
                                    track.data['ehpe'][i[0]] <= 50 and \
                                    track.data['ehve'][i[0]] <= 4:

                                lat = track.data['lat'][i[0]]
                                lon = track.data['lon'][i[0]]
                                point = Point(lat, lon)

                                if harbour.contains(point):
                                    sog =  track.data['sog'][i[0]]
                                    if sog > maxSog:
                                        maxSog = sog
                    
                        if maxSog > speedThreshold:
                            msg = '{:.02f},{:.02f},{:.02f}'.format(
                                maxSog, maxSog * 3600 / 1852, maxSog * 3600 / 1000)
                            stats[os.path.basename(filePath)] = msg
                            print('S', end='')
                        else:
                            print('.', end='')
                        
                        totPoints += len(track.data['sog'])
                        if ext in ['.oao']:
                            totDist += track.data['sog'].sum() / 5
                        else:
                            totDist += track.data['sog'].sum()

                    totFiles += 1

                except Exception:
                    errors[filePath.replace(projdir + '/', '')] = traceback.format_exc()
                    print('E', end='')

    print(os.linesep * 2 + 'Summary: files = {:,}, points = {:,}, distance = {:,.02f} km'.format(
        totFiles, totPoints, totDist / 1000))

    if len(errors) > 0:
        print(os.linesep * 2 + 'Errors:')
        for error in errors:
            print(error)
            print(errors[error])

    if len(stats) > 0:
        print(os.linesep * 2 + 'Members:')
        for filename in stats:
            print(filename + ',' + stats[filename])

In [3]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    findMembers()
    
    print(os.linesep + 'All done!')

.......................................................................S.................................................................................................................................................................................................................S......S....S........S...S................................S.....S...........................................S.........S............................................................................................................................................................................................................................................................................................................................................................S..S..................................S............S........................................................S...............................................................................................................................................